
# Configuring a Workforce Project

In this example, a new Workforce Project will be configured to manage some construction work that needs to be performed in Palm Springs. We will add assignment types, workers, dispatchers, and app integrations.

* Note: This requires using ArcGIS API for Python version 1.6 or higher.

### Import ArcGIS API for Python
Import the `arcgis` library and some modules within it.

In [ ]:
import arcgis
from arcgis.apps import workforce
from arcgis.gis import GIS

### Connect to the organization
Let's connect to our ArcGIS Online Organization.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")

### Fetch an Empty Workforce Project¶
Let's find the new Workforce Project.

In [ ]:
item = gis.content.get("29f2f056e5e04ba383e59737b3f8b340")
project = workforce.Project(item)

### Add Assignment Types
Let's add three assignment types related to road construction.

In [ ]:
fill_pothole = project.assignment_types.add(name="Fill in Pothole")
sidewalk_repair = project.assignment_types.add(name="Sidewalk Repair")
paint_crosswalk = project.assignment_types.add(name="Paint Crosswalk")

### Add Workers From an Existing Group
Let's add all of the construction workers that are already in an existing group.

In [ ]:
construction_crew = gis.groups.get("687cc3e68e8c4ef29dad20e8d6ebb9f5")
construction_crew

In [ ]:
for username in construction_crew.get_members()["users"]:
    user = gis.users.get(username)
    project.workers.add(
        user_id=username,
        name=user.fullName,
        status="not_working",
        title="Construction Worker"
    )

### Add Dispatchers From an Existing Group
Let's add all of the supervisors from an existing group of construction supervisors.

In [ ]:
construction_supervisors = gis.groups.get("55edd0e5a7d549cc99dd588d4ffa70f3")
construction_supervisors

In [ ]:
for username in construction_supervisors.get_members()["users"]:
    if username != "workforce_scripts":
        user = gis.users.get(username)
        project.dispatchers.add(
            user_id=username,
            name=user.fullName
        )

### Add Explorer Integration
Let's add an integration to open a specific map in Explorer for ArcGIS.

In [ ]:
from arcgis.apps import build_explorer_url
from arcgis.mapping import WebMap

webmap = WebMap(gis.content.get("268512553bbb4332ba0a864cbf4a5423"))
webmap.item.share(groups=[project.group])
webmap

We're going to use the `build_explorer_url` function to create the url that Workforce will use. In order for the integration to show up correctly in Workforce Back Office, we'll use a `url_type` of "App".

In [ ]:
url = build_explorer_url(
    webmap=webmap,
    center="${assignment.latitude},${assignment.longitude}",
    url_type="App",
    scale=9000
)

In [ ]:
project.integrations.add("default-explorer", "Explorer At Assignment", url)

### Add Waze Navigation Integration¶
Let's add a custom integration to allow workers to navigator to assignments using Waze.

In [ ]:
waze_url = "https://waze.com/ul?ll=${assignment.latitude},${assignment.longitude}&navigate=yes"

In [ ]:
project.integrations.add("waze-navigation", "Navigate Using Waze", waze_url)